In [326]:
# Import dependencies
import subprocess
import json
import os
import inspect
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit import *
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from bit import PrivateKeyTestnet
# from eth_account import Account
from getpass import getpass
from pathlib import Path
# from web3.middleware import geth_poa_middleware
from dotenv import load_dotenv

In [94]:
os.chdir(r'C:/Users/Lendl/Desktop/wallet')

In [ ]:
# chainID = w3.eth.chain_ID

In [184]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [146]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [134]:
# Import constants.py and necessary functions from bit and web3
from constants import BTC, ETH, BTCTEST

In [431]:
# Create a function called `derive_wallets`
def derive_wallets(phrase, coin, numderive):
    command = f'php derive -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --coin="{coin}" --numderive={numderive} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [432]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {ETH : derive_wallets(mnemonic, ETH, 3), BTCTEST : derive_wallets(mnemonic, BTCTEST, 3), BTC : derive_wallets(mnemonic, BTC, 5)}
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xfBb72CF96C633f79CFD1E92a8F2c27C3a39bEBD9',
   'privkey': '0x63ed7ad5b18599d37b1da7f5a0dbef46abff0ecbcf0a068f9e899ccdd325c5a5',
   'pubkey': '0343743f8b6373503ba39ff3a3a0de719e63077a1a7f3465458f1c2a783cb6f8db'},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x56F97cbe9794F95303E372A35445A791bb2c1CF6',
   'privkey': '0x08ae15496a0fb55edc6f2cbc7a0c51fe6fcfcdbf26e84ee1a170c78812272dda',
   'pubkey': '02c979f685bb405855a6f35a33b185d81bda8a185ba4e109e0814dbd76003ff844'},
  {'path': "m/44'/60'/0'/0/2",
   'address': '0xAb02eb178cc5Ca2bBaE41Ea7C8ffA86efa120673',
   'privkey': '0xbffddd765873887025f65db342282f78eed637c7c44cb86a8c3fc3b7dbc55f1f',
   'pubkey': '02e20d3f2610972f31e00e3074fd7c09db9fc3c0377b4bf1bd2003fbf05c7ad322'}],
 'btc-test': [{'path': "m/44'/1'/0'/0/0",
   'address': 'mtuwazor4ANd8YThSo2tCwaN2u8FUcXZ1s',
   'privkey': 'cW8hsBuQzg63Wame9JZr5E6jQBzwG14bfEyo6qA7XQvPss1JN3vh',
   'pubkey': '03911f4fca6f374693859f710cac1c25c70fd

In [264]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return bit.PrivateKeyTestnet(priv_key)


In [409]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient.address, "value": amount}
)
        return {
            "from": account.address,
            "to": recipient.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
#             "chainID": w3.eth.chainId
# Adding the "chainID":w3.eth.chainId" causes an error when sending a transaction
    }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient.address, amount, BTC)])


In [427]:
create_tx(BTCTEST, priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey']), priv_key_to_account(BTCTEST, coins[BTCTEST][1]['privkey']), 0.000001)

'{"unspents":[{"amount":58209,"confirmations":0,"script":"76a91492f29c855186c3d56cdb5dd099eecaa726b4c05388ac","txid":"4c64f789cb04def730df9e930cb26abaf37415c073fb7bb63832ef50204d1a54","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mk7Eut7ogLPtnmeviGwZkh56vx4z6BH33d",100],["mtuwazor4ANd8YThSo2tCwaN2u8FUcXZ1s",35057]]}'

In [396]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        raw_tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = NetworkAPI.broadcast_tx_testnet(signed_tx)
        print(signed_tx)
        return result

In [424]:
send_tx(BTCTEST, priv_key_to_account(BTCTEST,coins[BTCTEST][0]['privkey']), priv_key_to_account(BTCTEST,coins[BTCTEST][1]['privkey']), 0.000001)

0100000001f8b3a87abcdf2739b8a5d464064a0a22a391356a83efaff896568162c683c509010000006a47304402203969a73bc38b591be16a8a9de5781f6c83585a9c8b884f24819d6a9c6bb7b09e0220355b32ac6ca07ad44ae8dd66c0992bdfd0d15ff5b050f1c801775c83bba8f0da012103911f4fca6f374693859f710cac1c25c70fdb25516935331dba7456823cdcf35dffffffff0264000000000000001976a914325c8482703ab9e232ce55639a4ac632aed0e21788ac61e30000000000001976a91492f29c855186c3d56cdb5dd099eecaa726b4c05388ac00000000
